<a href="https://colab.research.google.com/github/FelipePinheiro964/PUCRS---Fase-2---Steam-Games/blob/main/PUCRS_Fase_2_Steam_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print(np.version)

<module 'numpy.version' from '/usr/local/lib/python3.10/dist-packages/numpy/version.py'>


*TRABALHO DE PROGRAMAÇÃO PARA DADOS*

_NOME:_ Felipe Pinheiro Fossá -
_CURSO:_ Banco de Dados: ênfase em data Analytics

In [57]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#seleção de arquivo
f = input('Digite o nome do arquivo: ')

#finalização forçada
if str.lower(f) == 'sair':
  print('Saindo...')
  sys.exit('O sistema foi finalizado pelo usuario, inicie novamente para outro uso.')
else:
  tabela = pd.read_csv(f, sep=",")
  if tabela.empty:
    print('Arquivo vazio.')

match input('Digite o número da questão: '):
  case '0':
    print('Saindo...')
    sys.exit('O sistema foi finalizado pelo usuario, inicie novamente para outro uso.')
  case '1': #questão 1
    try:

      questao_1 = tabela[['Name', 'Metacritic score', 'Release date']]

      if questao_1['Release date'].duplicated().any():
        questao_1_order = questao_1.sort_values(by= 'Release date' and 'Metacritic score', ascending=True)# Mostra os 10 maiores
        display(questao_1_order.tail(n=10))                                                               #false mostrava os 10 menores

      else:

        questao_1_10 = questao_1.copy()
        top_10_notas = questao_1_10.nlargest(10, 'Metacritic score').sort_values(by='Metacritic score', ascending=False) # mostra os maiores por ter poucos dados
        display(top_10_notas.tail(n=10))

    except FileNotFoundError:
      print('Digite um nome válido')

  case '2':
        #try:
          tabela_copy = tabela.copy() #cria uma copia da tabela
          mask = tabela_copy['Genres'].str.contains('RPG', case=False, na=False) #procura todos os generos RPG
          RPG = tabela_copy[mask] #cria uma variavel RPG para facilitar leitura
          RPG = RPG.dropna(subset=['Genres']) #retiro qualquer genero None presente
          print('Neste arquivo, temos:')
          display(RPG)
          #Media e maximo de DLCs no genero
          print()

          #DLCs do Genero RPG
          max_DLC = RPG['DLC count'].max() #mostra o maior valor de DLCs
          print(f'Para o genero RPG, temos {max_DLC:.2f} DLCs')

          media_DLC = RPG['DLC count'].mean()
          print(f'Em media, temos {media_DLC:.2f} DLCs presentes nesta categoria')

          print()

          #Media e maxima de avaliaçoes positivas
          max_positivas = RPG['Positive'].max() #mostra o maior valor de DLCs
          print(f'Para o genero RPG, temos {max_positivas:.2f} avalições positivas')

          media_positivas = RPG['Positive'].mean()
          print(f'Em media, temos {media_positivas:.2f} avalições positivas presentes nesta categoria')

          print()

          #Media e maxima de avaliaçoes positivas
          max_negativas = RPG['Negative'].max() #mostra o maior valor de DLCs
          print(f'Para o genero RPG, temos {max_negativas:.2f} avalições negativas')

          media_negativas = RPG['Negative'].mean()
          print(f'Em media, temos {media_negativas:.2f} avalições negativas presentes nesta categoria')
          #screenshots e filmes a ver


  case 3:


Digite o nome do arquivo: steam_games.csv
Digite o número da questão: 2
Neste arquivo, temos:


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
5,1469160,Wartune Reborn,"Feb 26, 2021",50000 - 100000,68,0,0.00,0,Feel tired of auto-fight? Feel tired of boring...,['English'],...,0,0,0,7Road,7Road,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip...","Turn-Based Combat,Massively Multiplayer,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
8,1178150,MazM: Jekyll and Hyde,"Apr 2, 2020",0 - 20000,1,0,14.99,0,'MazM: Jekyll and Hyde' is a darkly entertaini...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Growing Seeds,"CFK Co., Ltd.","Single-player,Steam Achievements,Full controll...","Adventure,RPG,Simulation,Strategy","Adventure,Simulation,RPG,Strategy,Singleplayer...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
10,1026420,WARSAW,"Oct 2, 2019",20000 - 50000,5,0,23.99,0,Use everything at your disposal to help a team...,"['English', 'French', 'German', 'Polish', 'Rus...",...,0,93,0,Pixelated Milk,"Pixelated Milk,gaming company","Single-player,Steam Achievements,Steam Trading...","Indie,RPG","Tactical RPG,Turn-Based Strategy,Wargame,Histo...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
14,1454010,Diary of Lucie,"Nov 25, 2020",0 - 20000,3,0,12.99,0,Diary of Lucie is action-roguelite game made w...,"['English', 'Korean']",...,0,0,0,Parange Project,Parange Project,"Single-player,Partial Controller Support,Steam...","Action,Adventure,Indie,RPG,Strategy,Early Access","Action Roguelike,Action,Rogue-lite,RPGMaker,My...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
17,346560,Hero of the Kingdom II,"Feb 20, 2015",100000 - 200000,5,0,7.99,0,Sail to the farthest islands to save your sist...,"['English', 'French', 'German', 'Spanish - Spa...",...,0,303,0,Lonely Troops,Lonely Troops,"Single-player,Steam Achievements,Steam Trading...","Adventure,Casual,Indie,RPG","Adventure,Casual,Point & Click,RPG,Indie,Isome...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72919,2168810,志怪者,"Apr 26, 2023",0 - 0,0,0,8.09,1,这是一场属于冒险者的游戏。“狂梦”危机席卷人类，作为调查员的你需要潜入梦中，通过调查与抉择、...,['Simplified Chinese'],...,0,0,0,RisingPunch Studio,RisingPunch Studio,"Single-player,Multi-player,Co-op,Online Co-op","RPG,Early Access",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
72921,2219140,Riboud and fairies,"Apr 25, 2023",0 - 20000,0,0,3.99,0,About this game Riboud and fairies is an excel...,"['English', 'Japanese', 'Simplified Chinese']",...,0,0,0,AI Leung,AI Leung,Single-player,"Action,Adventure,Casual,Indie,RPG","2D Platformer,Action RPG,Arcade,RPG,Action-Adv...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
72926,2365500,不被忘记的你,"Apr 28, 2023",0 - 0,0,0,0.00,0,角色 女主角，中文Galgame资料站CnGal的看板娘，有着人工智能的机器人，在CnGal...,['Simplified Chinese'],...,0,0,0,箱崎奈绪(Hakozaki Nao),箱崎奈绪(Hakozaki Nao),Single-player,"Adventure,Casual,Free to Play,Indie,RPG",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
72927,1245560,Roots of Pacha,"Apr 25, 2023",50000 - 100000,3346,0,22.49,0,Add Roots of Pacha to your wishlist! Join us o...,"['English', 'French', 'German', 'Portuguese - ...",...,420,598,494,Soda Den,Crytivo,"Single-player,Multi-player,Co-op,Online Co-op,...","Indie,RPG,Simulation","Pixel Graphics,Farming Sim,Life Sim,RPG,Romanc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.


Para o genero RPG, temos 2366.00 DLCs
Em media, temos 0.95 DLCs presentes nesta categoria


Para o genero RPG, temos 964983.00 avalições positivas
Em media, temos 1516.41 avalições positivas presentes nesta categoria

Para o genero RPG, temos 129925.00 avalições negativas
Em media, temos 247.17 avalições negativas presentes nesta categoria
